<img src="./images/kisti_logo.jpg" width="100"> <img src="./images/unist_logo.jpg" width="100"> <img src="./images/kisti_mas1.png" width="90"> <img src="./images/unist_mas.jpg" width="70">

# **2025년 겨울방학 AI 팀프로젝트 슈퍼컴퓨팅 청소년 캠프**
---
### **Edge Device를 이용한 AI 실습**
### **3. USB 카메라를 이용한 실시간 훈련 모델 확인**
---

In [ ]:
# 모델 경로와 클래스 정보 설정
MODEL_PATH = '/nvdli-nano/data/model_full.pth'
CATEGORIES = ['thumbs_down', 'thumbs_up']  # 카테고리 목록, 알파벳 순
#CATEGORIES = ['Paper', 'Rock', 'scissors']  # 카테고리 목록, 알파벳 순

---
### **카메라 설정 및 모델 로드**

In [ ]:
from jetcam.usb_camera import USBCamera
from jetcam.utils import bgr8_to_jpeg
import ipywidgets
import traitlets
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
from utils import preprocess
import threading

# 보드에 카메라 연결 확인
!ls -ltrh /dev/video*
# 카메라 설정 (USB 카메라 기준)
camera = USBCamera(width=320, height=240, capture_device=0)
camera.running = True
print("카메라 생성 완료")

# 모델 전체 로드 (구조 + 가중치)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 하드웨어 메시지 출력
if device.type == "cuda":
    print("cuda 이용")
else:
    print("cpu 이용")
    
model = torch.load(MODEL_PATH)
model = model.to(device)  # 모델을 GPU 또는 CPU로 이동
model.eval()

# display(model_widget)
print("모델 로드 및 설정 완료")
print(model)

---
### **<span style="color:red">모델의 결과를 위젯을 통해 실시간으로 확인</span>**
- category 별 막대 그래프를 통해 정확도 확인
- Prediction 값을 통해 카메라 이미지의 결과가 어떤 category에 포함되어 있는 지 확인
- Probabiligy 값을 통해 이미지가 포함된 category의 정확도를 확인

In [ ]:
# 위젯 생성
camera_widget = ipywidgets.Image()  # 실시간 카메라 이미지 표시# 카메라 화면 연결
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
prediction_widget = ipywidgets.Text(description='Prediction', value='', disabled=True)
probability_widget = ipywidgets.Text(description='Probability', value='', disabled=True)
score_widgets = [
    ipywidgets.FloatSlider(min=0.0, max=1.0, description=category, orientation='vertical')
    for category in CATEGORIES
]
state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='State', value='stop')

# 실시간 예측 함수
def live_prediction(state_widget, model, camera, prediction_widget, score_widgets):
    while state_widget.value == 'live':
        if camera.value is not None:
            image = preprocess(camera.value)  # 이미지 전처리
            output = model(image)
            output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()

            # 예측 결과 업데이트
            category_index = output.argmax()
            prediction_widget.value = CATEGORIES[category_index]
            probability_widget.value = f"{output[category_index] * 100:.2f}%"
            for i, score in enumerate(output):
                score_widgets[i].value = score

# 실시간 예측 시작 함수
def start_live(change):
    if change['new'] == 'live':
        threading.Thread(target=live_prediction, args=(state_widget, model, camera, prediction_widget, score_widgets)).start()

state_widget.observe(start_live, names='value')

# 위젯 레이아웃 구성
live_execution_widget = ipywidgets.VBox([
    ipywidgets.HBox(score_widgets),
    prediction_widget, probability_widget,
    state_widget
])

display(ipywidgets.VBox([
    ipywidgets.HBox([camera_widget]),
    live_execution_widget
]))

# 카메라 화면 연결
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
print("프로그램 실행 준비 완료")


---
### **카메라 및 프로그램 종료**

- 카메라 인스턴스 종료 및 커널 종료

In [ ]:
# 카메라 종료 (리소스 해제), 커널 종료
import os
os._exit(00)